Увидел у Игоря Викторовича в vk отличный пост: https://vk.com/wall137669108_516

![](https://pp.vk.me/c836220/v836220336/1fb0/GIzcsk2VWZ4.jpg)

In [1]:
import scipy.optimize
import numpy as np

In [2]:
coins = int(2 * 1e5)
gems = 115
mercury = 80
distant_min_power = 4000
air_min_power = 2000
gem_price = 500

X = [Титан, Нага, Джинн, Маг, Голем, Горгулья, Гремлин, Проданные камни]

In [3]:
loss_function = [-300, -120, -60, -40, -35, -20, -4, 0]

In [4]:
A = [[-300, 0, 0, -40, 0, 0, -4, 0], # distant
     [0, 0, -60, 0, 0, -20, 0, 0], # air
     [1, 0, 1, 1, 1, 0, 0, 0], # mercury
     [3, 2, 1, 1, 0, 0, 0, 1], # gems
     [5000, 1500, 750, 500, 400, 200, 70, -gem_price], # coins
     [1, 0, 0, 0, 0, 0, 0, 0], # titan
     [0, 1, 0, 0, 0, 0, 0, 0], # naga
     [0, 0, 1, 0, 0, 0, 0, 0], # djinn
     [0, 0, 0, 1, 0, 0, 0, 0], # mage
     [0, 0, 0, 0, 1, 0, 0, 0], # golem
     [0, 0, 0, 0, 0, 1, 0, 0], # gargoyle
     [0, 0, 0, 0, 0, 0, 1, 0], # gremlin
    ]
b = [-distant_min_power, -air_min_power, mercury, gems, coins, 10, 20, 30, 55, 60, 110, 500]

In [5]:
%%time
result = scipy.optimize.linprog(loss_function, A, b)

CPU times: user 20.9 ms, sys: 5.14 ms, total: 26 ms
Wall time: 22.7 ms


Решение найдено почти мгновенно.

In [6]:
result

     fun: -12875.0
 message: 'Optimization terminated successfully.'
     nit: 13
   slack: array([  1600.,   2000.,  23000.,      0.,      0.,      0.,     40.,
            0.,      0.,     35.,      0.,      0.])
  status: 0
 success: True
       x: array([  10.,   20.,   30.,   15.,   25.,  110.,  500.,    0.])

Последний элемент ответа 0 - камни продавать не нужно. Сила набранной армии: 12875.

Сила существ дальнего боя / воздушной армии:

In [7]:
-np.dot(result.x, np.array(A[0])), -np.dot(result.x, A[1])

(5600.0, 4000.0)

Затраченные ресурсы:

In [8]:
np.dot(result.x, A[4]), np.dot(result.x, A[3]), np.dot(result.x, A[2])

(177000.0, 115.0, 80.0)

Ртуть и камни потратили полностью, монеты еще остались.